<a href="https://colab.research.google.com/github/SohailVibeCoder/Olist-ML-project/blob/main/Foundations_of_Finance_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Import FF5 returns data
# Skiprows using nrows parameters to get the correct data
ff5_data = pd.read_csv("F-F_Research_Data_5_Factors_2x3.csv", skiprows=4, nrows=746)

# Rename first column to 'date'
ff5_data.rename(columns={ff5_data.columns[0]: 'date',
                         ff5_data.columns[1]: 'mktrf'}, inplace=True)

# Make all other columns lowercase
ff5_data.columns = [col.lower() for col in ff5_data.columns]

# Converting date to datetime format assuming that the data is valid as of the end of the month
ff5_data['date'] = pd.to_datetime(ff5_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)

# Import FF5 returns data
# Skiprows using nrows parameters to get the correct dataprint(ff5_data)

In [ ]:
# Import MoM Factors data
# Skiprows using nrows parameters to get the correct data

mom_data = pd.read_csv("F-F_Momentum_Factor.csv", skiprows=13, nrows=1184)
mom_data.rename(columns={mom_data.columns[0]: 'date',
                         mom_data.columns[1]: 'mom'}, inplace=True)


# Convert date to datetime format assuming that the data is valid as of the end of the month
mom_data['date'] = pd.to_datetime(mom_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)

In [ ]:
factor_data = pd.merge(ff5_data, mom_data, on='date', how='left')
factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']] = factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma','mom', 'rf']] / 100
factor_data.head()

,date,mktrf,smb,hml,rmw,cma,rf,mom
0,1963-07-31,-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027,0.0101
1,1963-08-31,0.0508,-0.0080,0.0170,0.0040,-0.0038,0.0025,0.0100
2,1963-09-30,-0.0157,-0.0043,0.0000,-0.0078,0.0015,0.0027,0.0012
3,1963-10-31,0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029,0.0313
4,1963-11-30,-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027,-0.0078


In [ ]:
# Filter the factor data from January 2015 to December 2024.

filtered_factor_data = factor_data[(factor_data['date'] >= '2015-01-01') & (factor_data['date'] <= '2024-12-31')].reset_index(drop=True)
display(filtered_factor_data.head())

#Verify that we have 120 rows in the filtered dataset
print(f"Shape of filtered data: {filtered_factor_data.shape}")

,date,mktrf,smb,hml,rmw,cma,rf,mom
0,2015-01-31,-0.0309,-0.0093,-0.0345,0.0158,-0.0164,0.0,0.0374
1,2015-02-28,0.0614,0.0036,-0.0179,-0.0110,-0.0175,0.0,-0.0310
2,2015-03-31,-0.0109,0.0308,-0.0038,0.0007,-0.0062,0.0,0.0270
3,2015-04-30,0.0060,-0.0301,0.0180,0.0005,-0.0062,0.0,-0.0727
4,2015-05-31,0.0138,0.0082,-0.0111,-0.0176,-0.0083,0.0,0.0568


Shape of filtered data: (120, 8)


In [ ]:
factors = ['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']

B1 = filtered_factor_data[factors].mean().reset_index()
B1.columns = ['Factors', 'Average Realized Monthly Return']

print("Table B1: Average Realized Monthly Return for Each Factor (2015-2024)")
display(B1)

Table B1: Average Realized Monthly Return for Each Factor (2015-2024)


,Factors,Average Realized Monthly Return
0,mktrf,0.009820
1,smb,-0.001664
2,hml,-0.001841
3,rmw,0.003903
4,cma,-0.000866
5,mom,0.002009
6,rf,0.001395


Question 2

In [ ]:
# Loading the CRSP Monthly Stock data
crsp_subset = pd.read_csv("crsp_monthly_subsample.csv")

In [ ]:
crsp_subset.head()

,permno,date,tsymbol,comnam,prc,ret,vol,shrout
0,10026,2015-01-30,JJSF,J & J SNACK FOODS CORP,98.12,-0.097913,15844.0,18688.0
1,10026,2015-02-27,JJSF,J & J SNACK FOODS CORP,101.19,0.031288,11316.0,18688.0
2,10026,2015-03-31,JJSF,J & J SNACK FOODS CORP,106.70,0.058010,19048.0,18689.0
3,10026,2015-04-30,JJSF,J & J SNACK FOODS CORP,104.33,-0.022212,11603.0,18691.0
4,10026,2015-05-29,JJSF,J & J SNACK FOODS CORP,107.80,0.033260,9415.0,18691.0


In [ ]:
# Convert date to datetime format
crsp_subset['date'] = pd.to_datetime(crsp_subset['date'])

# Clean price and return columns
crsp_subset['prc'] = crsp_subset['prc'].abs()
crsp_subset['ret'] = pd.to_numeric(crsp_subset['ret'], errors='coerce')
print(crsp_subset.shape)
crsp_subset.drop_duplicates(inplace=True)
print(crsp_subset.shape)

# Compute market capitalization
crsp_subset['mktcap'] = crsp_subset['prc'] * crsp_subset['shrout'] / 1e6  # in billions

(1222, 8)
(1200, 8)


In [ ]:
B2 = crsp_subset.groupby('permno').agg(
    mean_ret=('ret', 'mean')
).reset_index()

asset_name = crsp_subset.groupby('permno')['comnam'].last().reset_index()
B2 = B2.merge(asset_name, on='permno')
display(B2)

,permno,mean_ret,comnam
0,10026,0.006283,J & J SNACK FOODS CORP
1,10032,0.014102,PLEXUS CORP
2,10044,-0.006478,ROCKY MOUNTAIN CHOC FAC INC NEW
3,10065,0.012091,ADAMS EXPRESS CO
4,10104,0.014750,ORACLE CORP
5,10107,0.021769,MICROSOFT CORP
6,10318,0.010662,BALCHEM CORP
7,10333,0.017095,XOMA ROYALTY CORP
8,12079,0.010284,GENERAL MOTORS CO
9,21020,-0.001801,AMERICAN AIRLINES GROUP INC


Question 3

In [ ]:
# Merge on year and month
# - convert date to yearmonth variable
crsp_subset['yearmonth'] = crsp_subset['date'].dt.to_period('M')
filtered_factor_data['yearmonth'] = filtered_factor_data['date'].dt.to_period('M')

# - perform the merge
crsp_factors = pd.merge(crsp_subset, filtered_factor_data.drop(columns=['date']), on='yearmonth', how='left')
crsp_factors.head()

,permno,date,tsymbol,comnam,prc,ret,vol,shrout,mktcap,yearmonth,mktrf,smb,hml,rmw,cma,rf,mom
0,10026,2015-01-30,JJSF,J & J SNACK FOODS CORP,98.12,-0.097913,15844.0,18688.0,1.833667,2015-01,-0.0309,-0.0093,-0.0345,0.0158,-0.0164,0.0,0.0374
1,10026,2015-02-27,JJSF,J & J SNACK FOODS CORP,101.19,0.031288,11316.0,18688.0,1.891039,2015-02,0.0614,0.0036,-0.0179,-0.0110,-0.0175,0.0,-0.0310
2,10026,2015-03-31,JJSF,J & J SNACK FOODS CORP,106.70,0.058010,19048.0,18689.0,1.994116,2015-03,-0.0109,0.0308,-0.0038,0.0007,-0.0062,0.0,0.0270
3,10026,2015-04-30,JJSF,J & J SNACK FOODS CORP,104.33,-0.022212,11603.0,18691.0,1.950032,2015-04,0.0060,-0.0301,0.0180,0.0005,-0.0062,0.0,-0.0727
4,10026,2015-05-29,JJSF,J & J SNACK FOODS CORP,107.80,0.033260,9415.0,18691.0,2.014890,2015-05,0.0138,0.0082,-0.0111,-0.0176,-0.0083,0.0,0.0568


In [ ]:
# Compute excess returns for individual stocks
# - denote excess returns as 'retex'
crsp_factors['retex'] = crsp_factors['ret'] - crsp_factors['rf']

In [ ]:
B3 = crsp_factors.groupby('permno').agg(
    mean_retex=('retex', 'mean')
).reset_index()
B3

,permno,mean_retex
0,10026,0.004888
1,10032,0.012707
2,10044,-0.007873
3,10065,0.010696
4,10104,0.013355
5,10107,0.020374
6,10318,0.009267
7,10333,0.015700
8,12079,0.008889
9,21020,-0.003196


Question 4


In [ ]:
return_stats = crsp_factors.groupby('permno').agg(
    mean_ret=('ret', 'mean'),
    mean_retex=('retex', 'mean'),
    std_ret=('ret', 'std')
)

# Add the last comnam for each permno
last_comnam = crsp_factors.groupby('permno')['comnam'].last().reset_index()
return_stats = return_stats.merge(last_comnam, on='permno')
return_stats.columns = ['permno', 'mean_ret', 'mean_retex', 'std_ret', 'asset_name']

# Compute monthly Sharpe ratios of individual stocks,
return_stats['sharpe_ratio_MON'] = return_stats['mean_retex'] / return_stats['std_ret']

# Compute annualised Sharpe ratios
return_stats['sharpe_ratio_ANN'] = return_stats['sharpe_ratio_MON'] * np.sqrt(12)

B4 = return_stats[['permno','sharpe_ratio_ANN']]
B4.columns = ['permno', 'annualized_sharpe_ratio']

print("Table B4: Individual In-Sample Annualised Sharpe Ratio")
display(B4)

Table B4: Individual In-Sample Annualised Sharpe Ratio


,permno,annualized_sharpe_ratio
0,10026,0.264621
1,10032,0.573213
2,10044,-0.254606
3,10065,0.793359
4,10104,0.643786
5,10107,1.117247
6,10318,0.433011
7,10333,0.253458
8,12079,0.319075
9,21020,-0.093801


Question 5

In [ ]:
# Construct equally-weighted and value-weighted portfolios
# - ensure data is sorted by date and permno
crsp_factors = crsp_factors.sort_values(by=['date', 'permno'])

In [ ]:

# - equally weighted portfolio weights
crsp_factors['w_ew'] = 1 / crsp_factors.groupby('date')['permno'].transform('count')
crsp_factors['ret_ew'] = (crsp_factors['ret'] * crsp_factors['w_ew']).groupby(crsp_factors['date']).transform('sum')

portfolio_stats_ew = crsp_factors[(crsp_factors['date'] >= "2010-01-01") & \
                               (crsp_factors['date'] <= "2024-12-31")].groupby('date')['ret_ew'].first().reset_index()

# - compute mean and std dev
rf_mean = filtered_factor_data['rf'].mean()
mean_ew = portfolio_stats_ew['ret_ew'].mean()
ex_mean_ew = mean_ew - rf_mean
std_ew = portfolio_stats_ew['ret_ew'].std()

print(f"Average realized monthly excess return of the equally-weighted portfolio: {ex_mean_ew}")

portfolio_monthly_sharpe_ew = ex_mean_ew / std_ew
portfolio_annualized_sharpe_ew = portfolio_monthly_sharpe_ew * np.sqrt(12)

print(f"Annualized Sharpe ratio of the equally-weighted portfolio: {portfolio_annualized_sharpe_ew}")

Average realized monthly excess return of the equally-weighted portfolio: 0.00848051583333333
Annualized Sharpe ratio of the equally-weighted portfolio: 0.5379846446243807


Question 6


In [ ]:

# - get lagged market capitalization
crsp_factors['mktcap_l1'] = crsp_factors.groupby('permno')['mktcap'].shift(1)
# - value weighted portfolio weights
crsp_factors['w_vw'] = crsp_factors['mktcap_l1'] / crsp_factors.groupby('date')['mktcap_l1'].transform('sum')

B5 = crsp_factors[crsp_factors['yearmonth'] == '2017-01'][['permno','w_vw']].copy()
B5

,permno,w_vw
24,10026,0.003449
144,10032,0.002507
264,10044,0.000082
384,10065,0.001705
504,10104,0.218091
624,10107,0.664140
744,10318,0.003685
864,10333,0.000036
984,12079,0.073555
1104,21020,0.032749


Question 7


In [ ]:
crsp_factors['ret_vw'] = (crsp_factors['ret'] * crsp_factors['w_vw']).groupby(crsp_factors['date']).transform('sum')

portfolio_stats_vw = crsp_factors[(crsp_factors['date'] >= "2010-01-01") & \
                               (crsp_factors['date'] <= "2024-12-31")].groupby('date')['ret_vw'].first().reset_index()

# - compute mean and std dev
rf_mean = filtered_factor_data['rf'].mean()
mean_vw = portfolio_stats_vw['ret_vw'].mean()
ex_mean_vw = mean_vw - rf_mean
std_vw = portfolio_stats_vw['ret_vw'].std()

print(f"Average realized monthly excess return of the value-weighted portfolio: {ex_mean_vw:.6f}")

portfolio_monthly_sharpe_vw = ex_mean_vw / std_vw
portfolio_annualized_sharpe_vw = portfolio_monthly_sharpe_vw * np.sqrt(12)

print(f"Annualized Sharpe ratio of the value-weighted portfolio: {portfolio_annualized_sharpe_vw:.6f}")

Average realized monthly excess return of the value-weighted portfolio: 0.017881
Annualized Sharpe ratio of the value-weighted portfolio: 1.113153


Question 8

In [ ]:
print("Individual Stock Annualized Sharpe Ratios (Table B4):")
display(B4)

print(f"\nEqually-Weighted Portfolio Annualized Sharpe Ratio: {portfolio_annualized_sharpe_ew:.6f}")
print(f"Value-Weighted Portfolio Annualized Sharpe Ratio: {portfolio_annualized_sharpe_vw:.6f}")

print("\nDescriptive Statistics for Individual Stock Annualized Sharpe Ratios:")
display(B4['annualized_sharpe_ratio'].describe())

Individual Stock Annualized Sharpe Ratios (Table B4):


,permno,annualized_sharpe_ratio
0,10026,0.264621
1,10032,0.573213
2,10044,-0.254606
3,10065,0.793359
4,10104,0.643786
5,10107,1.117247
6,10318,0.433011
7,10333,0.253458
8,12079,0.319075
9,21020,-0.093801



Equally-Weighted Portfolio Annualized Sharpe Ratio: 0.537985
Value-Weighted Portfolio Annualized Sharpe Ratio: 1.113153

Descriptive Statistics for Individual Stock Annualized Sharpe Ratios:


,annualized_sharpe_ratio
count,10.000000
mean,0.404936
std,0.405265
min,-0.254606
25%,0.256249
50%,0.376043
75%,0.626143
max,1.117247


Question 9


In [ ]:
# Import libraries needed for linear regression
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

In [ ]:

# List of stocks: (permno, name)
stocks = {
    10107: "Microsoft",
    12079: "General Motors",
    21020: "American Airlines"
}

for permno, name in stocks.items():
    print("=" * 80)
    print(f"CAPM Regression Results – {name} (permno = {permno})")
    print("=" * 80)

    # Select stock
    reg_data = crsp_factors[crsp_factors['permno'] == permno]

    # Drop missing values
    reg_data = reg_data.dropna(subset=['retex', 'mktrf'])

    # Use full 120 months (assuming data already contains 120 months)
    X = add_constant(reg_data['mktrf'])
    y = reg_data['retex']

    print("X shape:", X.shape)
    print("y shape:", y.shape)

    # OLS with Newey-West standard errors
    model = OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': None})

    # Print regression output
    print(model.summary())


CAPM Regression Results – Microsoft (permno = 10107)
X shape: (120, 2)
y shape: (120,)
                            OLS Regression Results                            
Dep. Variable:                  retex   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     100.2
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           1.89e-17
Time:                        15:49:34   Log-Likelihood:                 197.52
No. Observations:                 120   AIC:                            -391.0
Df Residuals:                     118   BIC:                            -385.5
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------

Question 10


In [ ]:
from statsmodels.api import OLS, add_constant

# ============================
# FIGURE F2: FF 6-FACTOR MODEL
# ============================

stocks = {
    10107: "Microsoft",
    12079: "General Motors",
    21020: "American Airlines"
}

print("\nFIGURE F2: Fama-French 6-Factor Time-Series Regressions (120 Months)\n")

for permno, name in stocks.items():
    print("=" * 100)
    print(f"FIGURE F2 – FF 6-Factor Regression Output: {name} (permno = {permno})")
    print("=" * 100)

    # Select stock data
    reg_data = crsp_factors[crsp_factors['permno'] == permno]

    # Drop missing values
    reg_data = reg_data.dropna(
        subset=['retex','mktrf','smb','hml','rmw','cma','mom']
    )

    # Define dependent and independent variables
    X = add_constant(
        reg_data[['mktrf','smb','hml','rmw','cma','mom']]
    )
    y = reg_data['retex']

    print("X shape:", X.shape)
    print("y shape:", y.shape)

    # OLS with Newey-West standard errors
    model = OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': None})

    # Regression output (this IS Figure F2)
    print(model.summary())



FIGURE F2: Fama-French 6-Factor Time-Series Regressions (120 Months)

FIGURE F2 – FF 6-Factor Regression Output: Microsoft (permno = 10107)
X shape: (120, 7)
y shape: (120,)
                            OLS Regression Results                            
Dep. Variable:                  retex   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     66.66
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           7.80e-35
Time:                        15:49:36   Log-Likelihood:                 220.16
No. Observations:                 120   AIC:                            -426.3
Df Residuals:                     113   BIC:                            -406.8
Df Model:                           6                                         
Covariance Type:                  HAC                                         
                 coef    std err   